In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/codebert-base-finetuned-code-ner")
model = AutoModelForTokenClassification.from_pretrained("mrm8488/codebert-base-finetuned-code-ner")


/Users/kaylahuang/opt/anaconda3/envs/mercor/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:

# Create a NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Example code snippet
code_snippet = """
def calculate_average(numbers):
    total = sum(numbers)
    count = len(numbers)
    return total / count if count > 0 else 0

result = calculate_average([1, 2, 3, 4, 5])
print(f"The average is: {result}")
"""

# Perform NER on the code snippet
ner_results = ner_pipeline(code_snippet)

# Process and print the results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

# Function to highlight entities in the code
def highlight_entities(code, entities):
    highlighted = code
    offset = 0
    for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
        start = entity['start'] + offset
        end = entity['end'] + offset
        label = entity['entity_group']
        highlighted = highlighted[:start] + f"[{label}]{highlighted[start:end]}[/{label}]" + highlighted[end:]
        offset += len(label) * 2 + 5  # Adjust for the added characters

    return highlighted

# Highlight entities in the code
highlighted_code = highlight_entities(code_snippet, ner_results)
print("\nHighlighted Code:")
print(highlighted_code)

# Example of using the model for a specific task: Extracting function names
def extract_function_names(code):
    ner_results = ner_pipeline(code)
    function_names = [entity['word'] for entity in ner_results if entity['entity_group'] == 'method']
    return function_names

# Extract function names from the code snippet
functions = extract_function_names(code_snippet)
print("\nExtracted Function Names:")
print(functions)